In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


plt.rcParams['figure.figsize'] = [20, 5]

Reading and ploting the data

In [8]:
# Read the data file
DF = pd.read_csv("../data/public/train.csv", parse_dates = True)


DF['Day'] =  pd.to_datetime(DF['Day'], format='%Y-%m-%d')
DF.set_index("Day", inplace=True)
DF = DF.asfreq("D")
print(DF)


                 s001       s002       s003       s004       s005       s006  \
Day                                                                            
1996-03-18  13.407029  11.550454   5.640590  13.180272   9.778912   9.240363   
1996-03-19  14.725057  13.591270  14.399093   8.446712  10.813492  11.635488   
1996-03-20  20.564059  15.036848  24.418934  19.515306  21.612812  12.103175   
1996-03-21  34.708050  21.570295  28.784014  28.883220  38.520408  21.414399   
1996-03-22  26.629819  19.444444  20.620748  19.472789  24.744898  24.674036   
...               ...        ...        ...        ...        ...        ...   
1998-03-18  30.087868  18.750000        NaN  27.054989  25.283447  23.767007   
1998-03-19  54.138322  29.676871  21.683673  36.848073  41.723356  30.881519   
1998-03-20  53.500567  30.725624  35.586735  29.946145  33.475057  37.074830   
1998-03-21  39.696712  16.737528  18.636621  20.082200  16.198980   4.067460   
1998-03-22  29.705215  12.046485  13.095

In [9]:
print(DF.shape)

(735, 111)


Computing missing values, predicting and replacing them with an autoarima model

In [10]:
print(DF.isna().sum().sort_values(ascending=False))

s095    29
s064    28
s016    26
s078    23
s051    23
        ..
s027     9
s108     9
s010     9
s040     8
s032     8
Length: 111, dtype: int64


In [11]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error

for i in list(DF.columns):
    #Checking for missing values and storing them for later use
    missing_dates = DF[i][DF[i].isnull()].index

    # Split the data into a training set and a test set
    train_data = DF[i][~DF[i].index.isin(missing_dates)]
    test_data = DF[i][DF[i].index.isin(missing_dates)]

    # Use the auto_arima function to fit an ARIMA model to the training data
    model = auto_arima(train_data, error_action='ignore', suppress_warnings=True)

    # Forecast the missing values in the test set
    forecast = model.predict(n_periods=len(test_data))
    forecast = list(forecast)

    # Replace the missing values by the forecasted values 
    j = 0
    for date in missing_dates:
        DF[i][date] = forecast[j]
        j += 1

    # We could evaluate the models by comparing the forcasted values to the actual values
    #rmse = np.sqrt(mean_squared_error(forecast, test_data))
    #print("RMSE: ", rmse)

print(DF.isna().sum().sort_values(ascending=False))

c:\Users\teymu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\teymu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\teymu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\teymu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an

s001    0
s071    0
s082    0
s081    0
s080    0
       ..
s034    0
s033    0
s032    0
s031    0
s111    0
Length: 111, dtype: int64


c:\Users\teymu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Saving the dataset without missing values for further use in other notebooks

In [12]:
DF.to_csv('../data/public/clean_train.csv')